In [1]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple
import pandas as pd
from collections import defaultdict as _dd
import hashlib, math, bisect
import random as _rnd
from collections import Counter

# Constants 

In [2]:
SLOTS_PER_EPOCH   = 24000
EEP_FRACTION      = 1/3
EEP_SLOT          = int(SLOTS_PER_EPOCH * EEP_FRACTION)

PENALTY_FACTOR        = 2.0
RECOVERY_FACTOR       = 1.0
TOLERATED_BAD_RATIO   = 0.02
REWARD_PER_BLOCK      = 0.0001
AUTO_COMPOUND         = True
RNG_SEED              = 42
MIN_BET_AMOUNT        = 50.0
TOKEN_SCALE_INT       = 10**18
W_SKIPPED             = 1.0
W_INVALID             = 2.0
W_DELAYED             = 1.5

In [3]:
FEE_SCHEDULE_TKR: Dict[str, float] = {
    "PAY":                0.0001,
    "STAKE":              0.0010,
    "STAKE_UNDO":         0.0010,
    "REGISTER_MAIN":      1.0,
    "REGISTER_OVERFLOW":  0.1,
    "DEREGISTER_MAIN":    0.5,
    "DEREGISTER_OVERFLOW":0.05,
    "ASSIGN_OVERFLOW":    0.05,
    "UPLOAD_CONTRACT":    10.0,
    "CALL_FUNCTION":      0.01,
}

# Classes

In [4]:
@dataclass(slots=True)
class Wallet:
    address: str
    green: float = 0.0   # TKG
    red: float   = 0.0   # TKR

    def get_balance(self) -> Dict[str, float]:
        return {"green": self.green, "red": self.red}

    def deposit(self, *, green: float = 0.0, red: float = 0.0) -> None:
        if green > 0: self.green += green
        if red   > 0: self.red   += red

    def transfer(self, other: "Wallet", *, green_amount: float = 0.0, red_amount: float = 0.0) -> bool:
        if green_amount < 0 or red_amount < 0: return False
        if green_amount > self.green or red_amount > self.red: return False
        if green_amount:
            self.green -= green_amount; other.green += green_amount
        if red_amount:
            self.red   -= red_amount;   other.red   += red_amount
        return (green_amount > 0) or (red_amount > 0)

    def bet_op(self, to_main_id: str, amount: float, when_slot: int) -> dict:
        return {"kind": "STAKE", "from": self.address, "to": to_main_id,
                "amount": float(amount), "slot": when_slot}

In [5]:
@dataclass
class MainAddress(Wallet):
    delegates: Dict[str, float] = field(default_factory=dict)
    active_stake: float = 0.0
    overflows: Dict[str, "OverflowNode"] = field(default_factory=dict)

    def register_overflow(self, overflow: "OverflowNode") -> None:
        overflow.main_id = self.address
        self.overflows[overflow.address] = overflow

    def get_overflow_list(self) -> list:
        return list(self.overflows.values())

    def bet(self, from_address: str, amount: float) -> bool:
        if amount < MIN_BET_AMOUNT: return False
        self.delegates[from_address] = self.delegates.get(from_address, 0.0) + amount
        return True

    def self_bet(self, amount: float) -> bool:
        if amount < MIN_BET_AMOUNT or amount > self.green: return False
        self.green -= amount
        return self.bet("_self_", amount)

    def self_unbet(self, amount: Optional[float] = None) -> float:
        cur = self.delegates.get("_self_", 0.0)
        if cur <= 0: return 0.0
        take = cur if amount is None else max(0.0, min(amount, cur))
        if take <= 0: return 0.0
        residual = cur - take
        if residual > 0: self.delegates["_self_"] = residual
        else:            self.delegates.pop("_self_", None)
        self.green += take
        return take

    def total_delegated_now(self) -> float:
        return sum(self.delegates.values())

    def snapshot_at_eep(self) -> None:
        self.active_stake = self.total_delegated_now()

    def credit_reward(self, amount_green: float) -> None:
        """Coinbase in TKG pro-rata sulle deleghe (compounding)."""
        if amount_green <= 0: return
        total = self.total_delegated_now()
        if total <= 0:
            self.green += amount_green
            return
        for addr, stake_amt in list(self.delegates.items()):
            share = amount_green * (stake_amt / total)
            self.delegates[addr] = stake_amt + share

In [6]:
@dataclass(slots=True)
class OverflowNode(Wallet):
    main_id: str = ""
    # comportamento rete
    p_skip: float = 0.00
    p_invalid: float = 0.00
    p_delayed: float = 0.00
    # comportamento tx
    p_bad_tx: float = 0.0
    # stato
    penalty_score: float = 0.0
    effective_stake: float = 0.0
    assigned_slots: int = 0
    penalty_slots_residual: int = 0

    def behavior_outcome(self, rng) -> str:
        x = rng.random()
        if x < self.p_skip:    return "skipped"
        x -= self.p_skip
        if x < self.p_invalid: return "invalid"
        x -= self.p_invalid
        if x < self.p_delayed: return "delayed"
        return "ok"

    def slash_green(self, amount: float) -> float:
        if amount <= 0: return 0.0
        taken = min(self.green, amount)
        self.green -= taken
        return taken

In [7]:
@dataclass(slots=True)
class Transaction:
    txid: str
    kind: str
    sender: str
    recipient: str
    amount_green: float = 0.0
    amount_red: float = 0.0
    fee_paid_tkr: float = 0.0
    valid: bool = True

In [8]:
class TakamakaSim:
    def __init__(self,
                 *,
                 slots_per_epoch: int = SLOTS_PER_EPOCH,
                 eep_fraction: float = EEP_FRACTION,
                 penalty_factor: float = PENALTY_FACTOR,
                 recovery_factor: float = RECOVERY_FACTOR,
                 tolerated_bad_ratio: float = TOLERATED_BAD_RATIO,
                 reward_per_block: float = REWARD_PER_BLOCK,
                 seed: int = RNG_SEED,
                 slash_per_invalid: float = 0.0,
                 penalty_alpha: float = 1e-3,
                 w_skipped: float = W_SKIPPED,
                 w_invalid: float = W_INVALID,
                 w_delayed: float = W_DELAYED,
                 txs_per_slot_mean: int = 1,
                 max_txs_per_block: int = 1,
                 # scenario
                 tx_invalid_rate: float = 0.05,
                 auto_mempool_epoch0: bool = False,
                 auto_mempool_after_epoch0: bool = True,
                 malicious_invalid_share: float = 0.25,
                 # nuovi parametri (default ragionevoli)
                 pen_eps_low: float = 0.10,      # tolleranza -10% sotto gli slot attesi
                 pen_eps_high: float = 0.20,     # tolleranza +20% sopra gli slot attesi
                 pen_range_c_low: float = 0.25,  # peso penalità quando sotto-fai
                 pen_range_c_high: float = 0.75  # peso penalità quando sovra-fai
                 ):
        self.cfg = {
            "slots_per_epoch": int(slots_per_epoch),
            "eep_slot": int(slots_per_epoch * eep_fraction),
            "penalty_factor": float(penalty_factor),
            "recovery_factor": float(recovery_factor),
            "tolerated_bad_ratio": float(tolerated_bad_ratio),
            "reward_per_block": float(reward_per_block),
            "seed": int(seed),
            "slash_per_invalid": float(slash_per_invalid),
            "stake_scale_int": int(TOKEN_SCALE_INT),
            "penalty_alpha": float(penalty_alpha),
            "w_skipped": float(w_skipped),
            "w_invalid": float(w_invalid),
            "w_delayed": float(w_delayed),
            "txs_per_slot_mean": int(txs_per_slot_mean),
            "max_txs_per_block": int(max_txs_per_block),
            "tx_invalid_rate": float(tx_invalid_rate),
            "auto_mempool_epoch0": bool(auto_mempool_epoch0),
            "auto_mempool_after_epoch0": bool(auto_mempool_after_epoch0),
            "malicious_invalid_share": float(malicious_invalid_share),
            "pen_eps_low": float(pen_eps_low),
            "pen_eps_high": float(pen_eps_high),
            "pen_range_c_low": float(pen_range_c_low),
            "pen_range_c_high": float(pen_range_c_high),
        }

        self.epoch = 0
        self.mains: Dict[str, MainAddress] = {}
        self.overflows: Dict[str, OverflowNode] = {}
        self.wallets: Dict[str, Wallet] = {}

        self.pending_ops: List[Dict] = []
        self.history: List[Dict] = []

        self.weights_for_epoch: Dict[str, float] = {}
        self.weights_next_epoch: Dict[str, float] = {}

        self.current_schedule: List[str] = []
        self.next_schedule: List[str] = []

        self.current_seed: bytes = b""
        self.next_seed: bytes = b""

        self._slot_outcomes_current: List[str] = []
        self._initialized_weights = False

        self.mempool: List[Transaction] = []
        self._tx_seq = 0

        self.tx_senders_whitelist: Optional[set[str]] = None
        self.tx_recipients_whitelist: Optional[set[str]] = None
        self.tx_min_fee_kind: str = "PAY"

    # --------------------------- Registrazione
    def register_wallet(self, w: Wallet): self.wallets[w.address] = w
    def register_main(self, m: MainAddress): self.mains[m.address] = m
    def index_overflow(self, node: OverflowNode):
        if node.main_id not in self.mains:
            raise ValueError("main_id non registrato")
        self.overflows[node.address] = node

    # --------------------------- Operazioni differite (EEP)
    def queue_bet(self, from_addr: str, to_main_id: str, amount: float, when_slot: int):
        if to_main_id not in self.mains:
            raise ValueError(f"Main non registrato: {to_main_id}")
        if from_addr not in self.wallets and from_addr != "_self_":
            raise ValueError(f"Stakeholder non registrato: {from_addr}")
        if amount < MIN_BET_AMOUNT:
            raise ValueError(f"Importo minimo di puntata è {MIN_BET_AMOUNT}, got {amount}")
        self.pending_ops.append({
            "kind": "STAKE", "from": from_addr, "to": to_main_id,
            "amount": float(amount), "epoch": self.epoch, "slot": int(when_slot),
        })

    def queue_stake_undo(self, from_addr: str, when_slot: int):
        if from_addr not in self.wallets:
            raise ValueError(f"Stakeholder non registrato: {from_addr}")
        self.pending_ops.append({
            "kind": "STAKE_UNDO", "from": from_addr,
            "epoch": self.epoch, "slot": int(when_slot),
        })

    def queue_self_unbet(self, main_id: str, when_slot: int, amount: Optional[float] = None):
        if main_id not in self.mains:
            raise ValueError(f"Main non registrato: {main_id}")
        self.pending_ops.append({
            "kind": "SELF_UNBET", "to": main_id,
            "amount": None if amount is None else float(amount),
            "epoch": self.epoch, "slot": int(when_slot),
        })

    def _apply_ops_at_current_eep(self):
        eep = self.cfg["eep_slot"]; ep = self.epoch
        eligible, keep = [], []
        for op in self.pending_ops:
            if (op["epoch"] == ep and op["slot"] <= eep) or (op["epoch"] == ep - 1 and op["slot"] > eep):
                eligible.append(op)
            else:
                keep.append(op)

        eligible.sort(key=lambda op: (
            0 if op["kind"] in ("STAKE_UNDO", "SELF_UNBET") else 1,
            op.get("epoch", -1), op.get("slot", -1), op.get("from", ""), op.get("to", "")
        ))

        for op in eligible:
            k = op["kind"]
            if k == "STAKE_UNDO":
                st = self.wallets.get(op["from"])
                if not st: continue
                for m in self.mains.values():
                    cur = m.delegates.get(op["from"], 0.0)
                    if cur > 0:
                        st.green += cur
                        m.delegates.pop(op["from"], None)
            elif k == "STAKE":
                to_main = self.mains.get(op["to"])
                if not to_main: continue
                amt = float(op["amount"])
                if amt < MIN_BET_AMOUNT: continue
                if op["from"] == "_self_":
                    if amt <= to_main.green:
                        to_main.green -= amt
                        to_main.bet("_self_", amt)
                else:
                    st = self.wallets.get(op["from"])
                    if st and amt <= st.green:
                        st.green -= amt
                        to_main.bet(op["from"], amt)
            elif k == "SELF_UNBET":
                to_main = self.mains.get(op["to"])
                if to_main:
                    to_main.self_unbet(op.get("amount", None))

        self.pending_ops = keep

    def _snapshot_all_mains_at_eep(self):
        for m in self.mains.values():
            m.snapshot_at_eep()

    # --------------------------- Seed & VRF
    @staticmethod
    def _int_addr(addr: str) -> int:
        return int.from_bytes(hashlib.sha256(addr.encode()).digest(), "big", signed=False)

    def _build_intervals_from_weights_vrf(self, weights: Dict[str, float]) -> Tuple[List[str], List[int], int]:
        scale = int(self.cfg["stake_scale_int"])
        items = []
        for addr, w in weights.items():
            wint = int((0.0 if w is None else float(w)) * scale)
            if wint > 0:
                items.append((addr, self._int_addr(addr), wint))
        items.sort(key=lambda x: x[1])
        addresses, prefix_ends, acc = [], [], 0
        for addr, _, wint in items:
            addresses.append(addr); acc += wint; prefix_ends.append(acc)
        return addresses, prefix_ends, acc

    @staticmethod
    def _hash_chain(seed: bytes, n: int):
        x = hashlib.sha256(seed).digest()
        for _ in range(n):
            yield x
            x = hashlib.sha256(x).digest()

    def _vrf_schedule_exact(self, seed: bytes, weights: Dict[str, float]) -> Tuple[Dict[str, int], List[str]]:
        S = self.cfg["slots_per_epoch"]
        addrs, prefix_ends, total_int = self._build_intervals_from_weights_vrf(weights)
        if total_int <= 0 or not addrs:
            return {addr: 0 for addr in self.overflows}, []
        counts = {addr: 0 for addr in self.overflows}
        schedule: List[str] = []
        for x in self._hash_chain(seed, S):
            ticket = int.from_bytes(x, "big") % total_int
            j = bisect.bisect_left(prefix_ends, ticket)
            if j >= len(addrs): j = len(addrs) - 1
            addr = addrs[j]
            schedule.append(addr)
            counts[addr] = counts.get(addr, 0) + 1
        return counts, schedule

    # --------------------------- Pesi effettivi
    def _compute_effective_weights_now(self) -> Dict[str, float]:
        weights: Dict[str, float] = {}
        alpha = self.cfg["penalty_alpha"]
        for addr, node in self.overflows.items():
            main = self.mains[node.main_id]
            resid = max(0, int(getattr(node, "penalty_slots_residual", 0)))
            node.penalty_score = alpha * resid
            weights[addr] = main.active_stake / (1.0 + node.penalty_score)
        return weights

    # --------------------------- Penalità
    def _apply_penalties(self, per_node_outcomes: Dict[str, Dict[str, int]]) -> Dict[str, Dict[str, float]]:
        wS = self.cfg["w_skipped"]; wI = self.cfg["w_invalid"]; wD = self.cfg["w_delayed"]
        PF = self.cfg["penalty_factor"]; RF = self.cfg["recovery_factor"]
        tol_ratio = self.cfg["tolerated_bad_ratio"]; slash = self.cfg["slash_per_invalid"]
        alpha = self.cfg["penalty_alpha"]

        # --- NUOVO: finestra attesa per nodo basata sui pesi correnti (epoch in corso)
        # usa i pesi effettivi già calcolati e “freezati” per QUESTA epoch
        weights_cur = {addr: max(0.0, self.overflows[addr].effective_stake) for addr in self.overflows}
        tot_w = sum(weights_cur.values()) or 1.0
        E_S = self.cfg["slots_per_epoch"]

        eps_low  = self.cfg["pen_eps_low"]
        eps_high = self.cfg["pen_eps_high"]
        C_LOW    = self.cfg["pen_range_c_low"]
        C_HIGH   = self.cfg["pen_range_c_high"]

        debug: Dict[str, Dict[str, float]] = {}

        for addr, outcome in per_node_outcomes.items():
            node = self.overflows[addr]
            assigned = max(1, int(getattr(node, "assigned_slots", 0)))
            ok      = int(outcome.get("ok", 0))
            skipped = int(outcome.get("skipped", 0))
            invalid = int(outcome.get("invalid", 0))
            delayed = int(outcome.get("delayed", 0))

            # --- (A) Penalità qualità esiti con tolleranza
            tol = int(math.floor(tol_ratio * assigned))
            bad_weighted = (wS * skipped) + (wI * invalid) + (wD * delayed)
            gained_A = max(0.0, bad_weighted - tol) * PF

            # --- (B) Penalità “capacità” fuori finestra attesa (asimmetrica)
            # slot attesi μ_i = E_S * (w_i / Σ w_j)
            mu_i = E_S * (weights_cur[addr] / tot_w)
            low_i  = (1.0 - eps_low)  * mu_i
            high_i = (1.0 + eps_high) * mu_i

            excess_low  = max(0.0, low_i  - assigned)   # sotto-produzione
            excess_high = max(0.0, assigned - high_i)   # sovra-produzione

            gained_B = C_LOW * excess_low + C_HIGH * excess_high

            # totale guadagnato questa epoch
            gained = int(math.ceil(gained_A + gained_B))

            # recovery lento sui "buoni"
            recovery = int(math.floor(RF * ok))

            # aggiorna residuo
            resid_prev = max(0, int(getattr(node, "penalty_slots_residual", 0)))
            resid_next = max(0, resid_prev + gained - recovery)
            node.penalty_slots_residual = resid_next
            node.penalty_score = alpha * resid_next

            # opzionale: slashing TKG per invalid (se vuoi tenerlo)
            if slash > 0.0 and invalid > 0:
                node.slash_green(invalid * slash)

            debug[addr] = {
                "assigned": assigned, "ok": ok, "skipped": skipped, "invalid": invalid, "delayed": delayed,
                "mu_i": float(mu_i), "low_i": float(low_i), "high_i": float(high_i),
                "bad_weighted": float(bad_weighted), "tol": float(tol),
                "gained_A": float(gained_A), "gained_B": float(gained_B),
                "gained": float(gained), "recovery": float(recovery),
                "residual_prev": float(resid_prev), "residual_next": float(resid_next),
            }

        return debug


    # --------------------------- Mempool & TX
    def _wallet_by_addr(self, addr: str) -> Optional[Wallet]:
        return self.wallets.get(addr) or self.mains.get(addr) or self.overflows.get(addr)

    def set_tx_participants(self, *, senders: Optional[list[str]] = None,
                            recipients: Optional[list[str]] = None) -> None:
        self.tx_senders_whitelist = set(senders) if senders else None
        self.tx_recipients_whitelist = set(recipients) if recipients else None

    def _eligible_parties(self) -> Tuple[List[str], List[str]]:
        fee_min = float(FEE_SCHEDULE_TKR.get(self.tx_min_fee_kind, 0.0))
        base_s = self.tx_senders_whitelist or set(self.wallets.keys())
        base_r = self.tx_recipients_whitelist or set(self.wallets.keys())
        senders = [a for a in base_s if (w := self.wallets.get(a)) and (w.red + w.green) >= fee_min]
        recipients = [a for a in base_r if a in self.wallets]
        return senders, recipients

    def _gen_tx_batch(self, rng, n: int, *, allow_invalids: bool) -> List[Transaction]:
        if n <= 0: return []
        senders, recipients = self._eligible_parties()
        if not senders or not recipients: return []
        required_fee = float(FEE_SCHEDULE_TKR.get(self.tx_min_fee_kind, 0.0))
        txs = []; choice = rng.choice; rnd = rng.random
        for _ in range(n):
            s = choice(senders); t = choice(recipients)
            tries = 0
            while t == s and tries < 3: t = choice(recipients); tries += 1
            if s == t: continue
            wsrc = self.wallets[s]
            amt_g = (rnd() * min(5.0, wsrc.green)) if rnd() < 0.3 else 0.0
            amt_r = (rnd() * min(5.0, wsrc.red))   if rnd() < 0.7 else 0.0
            txid = f"{self.epoch}:{self._tx_seq}"; self._tx_seq += 1
            tx = Transaction(txid=txid, kind="PAY", sender=s, recipient=t,
                             amount_green=float(amt_g), amount_red=float(amt_r), valid=True)
            tx.fee_paid_tkr = required_fee
            if allow_invalids and (rnd() < self.cfg["tx_invalid_rate"]):
                if rnd() < 0.5:
                    tx.valid = False
                else:
                    tx.valid = True
                    tx.fee_paid_tkr = required_fee * 0.5
            txs.append(tx)
        return txs

    @staticmethod
    def _calc_required_fee_tkr(tx: Transaction) -> float:
        return float(FEE_SCHEDULE_TKR.get(tx.kind, 0.0))

    def _pay_fee_and_apply_payload(self, tx: Transaction, *, dry_run: bool = False) -> bool:
        src = self._wallet_by_addr(tx.sender); dst = self._wallet_by_addr(tx.recipient)
        if not src or not dst: return False
        required = self._calc_required_fee_tkr(tx)
        paid = float(getattr(tx, "fee_paid_tkr", 0.0))
        if paid + 1e-18 < required: return False

        need = required
        if dry_run:
            can_r = min(src.red, need); need -= can_r
            if need > 0: can_g = min(src.green, need); need -= can_g
            if need > 0: return False
        else:
            take_r = min(src.red, need); src.red -= take_r; need -= take_r
            if need > 0:
                take_g = min(src.green, need); src.green -= take_g; need -= take_g
            if need > 0: return False

        if tx.kind == "PAY":
            if src.green < tx.amount_green or src.red < tx.amount_red:
                return False
            if not dry_run:
                if tx.amount_green > 0: src.green -= tx.amount_green; dst.green += tx.amount_green
                if tx.amount_red   > 0: src.red   -= tx.amount_red;   dst.red   += tx.amount_red
        return True

    # --------------------------- Next seed/schedule
    def _compute_next_seed_from_eep_block(self) -> bytes:
        eep_idx = max(0, self.cfg["eep_slot"] - 1)
        chosen_slot = None
        if self._slot_outcomes_current:
            i = min(eep_idx, len(self._slot_outcomes_current) - 1)
            while i >= 0:
                if self._slot_outcomes_current[i] == "ok":
                    chosen_slot = i; break
                i -= 1
        if chosen_slot is None:
            return hashlib.sha256(b"fallback|seed|" + str(self.epoch).encode()).digest()
        producer_addr = self.current_schedule[chosen_slot]
        msg = (b"|epoch|" + self.epoch.to_bytes(8, "big") +
               b"|slot|" + chosen_slot.to_bytes(8, "big") +
               b"|producer|" + producer_addr.encode())
        return hashlib.sha256(msg).digest()

    def _prepare_next_epoch_state(self):
        self._apply_ops_at_current_eep()
        self._snapshot_all_mains_at_eep()
        self.weights_next_epoch = self._compute_effective_weights_now()
        self.next_seed = self._compute_next_seed_from_eep_block()
        _, self.next_schedule = self._vrf_schedule_exact(self.next_seed, self.weights_next_epoch)

    # --------------------------- Simulazione epoca
    def simulate_one_epoch(self) -> Dict:
        from collections import Counter

        cfg = self.cfg
        S = cfg["slots_per_epoch"]
        take_n = cfg["max_txs_per_block"]

        # ---------------- bootstrap epoca 0 ----------------
        if self.epoch == 0 and not self._initialized_weights:
            self._apply_ops_at_current_eep()
            self._snapshot_all_mains_at_eep()
            self.weights_for_epoch = self._compute_effective_weights_now()
            self._initialized_weights = True
            self.current_seed = hashlib.sha256(b"genesis-seed|" + str(cfg['seed']).encode()).digest()
            _, self.current_schedule = self._vrf_schedule_exact(self.current_seed, self.weights_for_epoch)

        # effective stake (per report di QUEST'epoca, cioè pre-penalità)
        for addr, node in self.overflows.items():
            node.effective_stake = float(self.weights_for_epoch.get(addr, 0.0))

        # ---------------- contatori per-epoca ----------------
        per_node_outcomes = _dd(lambda: {"ok": 0, "skipped": 0, "invalid": 0, "delayed": 0})
        fees_epoch_by_main = _dd(float)
        self._slot_outcomes_current = []
        slots_per_node = {addr: 0 for addr in self.overflows}

        rng_slots = _rnd.Random(cfg["seed"] * 7919 + self.epoch)

        # ---------------- mempool (epoca 0: niente auto; >=1: opzionale) ----------------
        self.mempool = []
        cap = S * take_n
        target = min(cap, S * cfg["txs_per_slot_mean"])
        if self.epoch >= 1 and cfg["auto_mempool_after_epoch0"]:
            left = int(target); BATCH = 5000
            while left > 0:
                n = BATCH if left > BATCH else left
                self.mempool.extend(self._gen_tx_batch(rng_slots, n, allow_invalids=True))
                left -= n
        mp = self.mempool

        # ---------------- budget invalid forzati per nodo/epoca ----------------
        forced_invalid_slots: Dict[str, set] = {addr: set() for addr in self.overflows}
        forced_invalid_count_planned: Dict[str, int] = {addr: 0 for addr in self.overflows}
        forced_invalid_count_realized: Dict[str, int] = {addr: 0 for addr in self.overflows}

        if self.epoch >= 1:
            planned_assign = Counter(self.current_schedule)
            share = float(cfg["malicious_invalid_share"])

            per_overflow_slot_indices: Dict[str, List[int]] = {addr: [] for addr in self.overflows}
            for idx, prod_addr in enumerate(self.current_schedule):
                per_overflow_slot_indices[prod_addr].append(idx)

            rng_force = _rnd.Random(cfg["seed"] * 104729 + self.epoch)
            for addr, node in self.overflows.items():
                if getattr(node, "p_bad_tx", 0.0) > 0.0:
                    assigned_now = planned_assign.get(addr, 0)
                    budget = int(math.floor(share * assigned_now))
                    slot_list = per_overflow_slot_indices.get(addr, [])
                    if budget > 0 and slot_list:
                        budget = min(budget, len(slot_list))
                        forced = rng_force.sample(slot_list, budget)
                        forced_invalid_slots[addr].update(forced)

            for addr, slset in forced_invalid_slots.items():
                forced_invalid_count_planned[addr] = len(slset)

        # ---------------- loop sugli slot ----------------
        for slot_idx, addr in enumerate(self.current_schedule):
            node = self.overflows[addr]
            main = self.mains[node.main_id]
            slots_per_node[addr] = slots_per_node.get(addr, 0) + 1

            # outcome base (solo condizioni di rete)
            if self.epoch == 0:
                outcome = "ok"  # epoca 0: tutti ok
            else:
                r = rng_slots.random()
                if r < node.p_skip:
                    outcome = "skipped"
                elif r < node.p_skip + node.p_invalid:
                    outcome = "invalid"  # invalid "di rete" (se abiliti p_invalid)
                elif r < node.p_skip + node.p_invalid + node.p_delayed:
                    outcome = "delayed"
                else:
                    outcome = "ok"

            # invalidazione forzata da comportamento malevolo (quota per epoca)
            force_invalid = (self.epoch >= 1) and (slot_idx in forced_invalid_slots.get(addr, set()))
            if force_invalid:
                # short-circuit: NON tocco stato, NON consumo mempool
                per_node_outcomes[addr]["invalid"] += 1
                forced_invalid_count_realized[addr] += 1
                self._slot_outcomes_current.append("invalid")
                continue

            # --- Provo a produrre solo se outcome "ok"
            if outcome == "ok":
                collected_fees_tkr = 0.0

                # prendo il batch SOLO qui (se c'è chance di chiudere ok)
                picked: List[Transaction] = mp[:take_n]

                # ---- 2-phase commit: DRY-RUN dell'intero batch, ma accetto il sottoinsieme valido
                prepared: List[Tuple[Transaction, float]] = []
                for tx in picked:
                    required = self._calc_required_fee_tkr(tx)
                    fee_paid = float(getattr(tx, "fee_paid_tkr", required))
                    formally_valid = getattr(tx, "valid", True) and (fee_paid + 1e-18 >= required)
                    if formally_valid and self._pay_fee_and_apply_payload(tx, dry_run=True):
                        prepared.append((tx, required))
                    # se una tx non passa, semplicemente NON la includo; il blocco può essere vuoto

                # ---- COMMIT delle sole prepared (se ce ne sono)
                can_commit = all(self._pay_fee_and_apply_payload(tx, dry_run=True) for tx, _ in prepared)
                commit_ok = True
                if can_commit and prepared:
                    for tx, required in prepared:
                        ok_commit = self._pay_fee_and_apply_payload(tx, dry_run=False)
                        if not ok_commit:
                            commit_ok = False
                            break
                        collected_fees_tkr += required
                # Nota: anche con prepared vuoto, commit_ok resta True → blocco vuoto ma valido

                if commit_ok:
                    # rimuovo ORA le tx consumate dal mempool
                    del mp[:len(picked)]
                    # reward + fee al main (fee solo se >0)
                    main.credit_reward(cfg["reward_per_block"])
                    if collected_fees_tkr > 0.0:
                        main.deposit(red=collected_fees_tkr)
                        fees_epoch_by_main[main.address] += collected_fees_tkr
                    per_node_outcomes[addr]["ok"] += 1
                    self._slot_outcomes_current.append("ok")
                else:
                    # rollback logico: nessun consumo mempool (non abbiamo cancellato nulla)
                    per_node_outcomes[addr]["invalid"] += 1
                    self._slot_outcomes_current.append("invalid")

            else:
                # outcome = skipped/delayed/invalid(di rete): NON consumare mempool, nessun reward
                per_node_outcomes[addr][outcome] += 1
                self._slot_outcomes_current.append(outcome)

        # ---------------- assegnazioni e invariante ----------------
        for addr, node in self.overflows.items():
            node.assigned_slots = slots_per_node.get(addr, 0)

        for addr in self.overflows:
            o = per_node_outcomes[addr]
            produced = o["ok"] + o["skipped"] + o["invalid"] + o["delayed"]
            assigned = slots_per_node.get(addr, 0)
            if produced != assigned:
                raise RuntimeError(
                    f"Slot tally mismatch for {addr}: produced={produced}, assigned={assigned}"
                )

        # ---------------- penalità e PREPARAZIONE next-epoch ----------------
        penalty_debug = self._apply_penalties(per_node_outcomes)

        # Calcola ORA seed/schedule/pesi next-epoch
        self._prepare_next_epoch_state()

        # ---------------- KPI/report snapshot dell'epoca corrente ----------------
        total_effective_stake = sum(max(0.0, v) for v in self.weights_for_epoch.values()) or 1.0
        effective_stake_next_epoch = {addr: float(self.weights_next_epoch.get(addr, 0.0))
                                    for addr in self.overflows}
        total_effective_stake_next = sum(max(0.0, v) for v in effective_stake_next_epoch.values()) or 1.0

        kpi = {
            "epoch": self.epoch,
            "slots_per_node": dict(slots_per_node),
            "per_node_outcomes": {addr: dict(v) for addr, v in per_node_outcomes.items()},
            "penalty_slots_residual": {addr: int(self.overflows[addr].penalty_slots_residual) for addr in self.overflows},
            "penalties_alpha_scaled": {addr: float(self.overflows[addr].penalty_score) for addr in self.overflows},

            # pesi di QUESTA epoca (usati per la schedule corrente)
            "effective_stake_per_overflow": {addr: float(self.overflows[addr].effective_stake) for addr in self.overflows},
            "total_effective_stake": float(total_effective_stake),

            # pesi della PROSSIMA epoca (dopo penalità)
            "effective_stake_next_epoch": effective_stake_next_epoch,
            "total_effective_stake_next": float(total_effective_stake_next),

            "total_slots": int(cfg["slots_per_epoch"]),
            "fees_epoch_by_main": dict(fees_epoch_by_main),
            "penalty_debug": penalty_debug,

            # breakdown invalid malevoli
            "forced_invalid_slots_count_planned": dict(forced_invalid_count_planned),
            "forced_invalid_slots_count_realized": dict(forced_invalid_count_realized),
        }
        self.history.append(kpi)

        # rollover → epoca successiva
        self.weights_for_epoch = dict(self.weights_next_epoch)
        self.current_seed = self.next_seed
        self.current_schedule = list(self.next_schedule)
        self.epoch += 1
        return kpi



    # --------------------------- Report
    def summary_all_epochs_df(self) -> pd.DataFrame:
        ok_cum_map = _dd(int)
        fees_cum_by_main = _dd(float)
        rows: List[Dict[str, object]] = []

        S = self.cfg["slots_per_epoch"]
        reward_per_block = float(self.cfg.get("reward_per_block", 0.0))

        for kpi in self.history:
            # snapshot (non stato vivo)
            total_effective_cur  = float(kpi.get("total_effective_stake", 0.0)) or 1.0
            total_effective_next = float(kpi.get("total_effective_stake_next", 0.0)) or 1.0
            total_slots_epoch    = int(kpi.get("total_slots", 0))

            kpi_outcomes   = kpi.get("per_node_outcomes", {})
            kpi_slots      = kpi.get("slots_per_node", {})
            kpi_eff_cur    = kpi.get("effective_stake_per_overflow", {})
            kpi_eff_next   = kpi.get("effective_stake_next_epoch", {})
            kpi_fees_m     = kpi.get("fees_epoch_by_main", {})
            kpi_resid      = kpi.get("penalty_slots_residual", {})
            kpi_score      = kpi.get("penalties_alpha_scaled", {})
            kpi_pdebug     = kpi.get("penalty_debug", {})

            # invalid malevoli (pianificati vs realizzati)
            kpi_forced_planned  = kpi.get("forced_invalid_slots_count_planned", {})
            kpi_forced_realized = kpi.get("forced_invalid_slots_count_realized", {})

            # cumulati per coinbase/fee
            for addr, o in kpi_outcomes.items():
                ok_cum_map[addr] += int(o.get("ok", 0))
            for m, f in kpi_fees_m.items():
                fees_cum_by_main[m] += float(f)

            # una riga per overflow nell'epoca
            for addr, slots in kpi_slots.items():
                o = kpi_outcomes.get(addr, {})
                ok      = int(o.get("ok", 0))
                skipped = int(o.get("skipped", 0))
                invalid = int(o.get("invalid", 0))
                delayed = int(o.get("delayed", 0))
                bad_abs = skipped + invalid + delayed

                node_eff_cur  = float(kpi_eff_cur.get(addr, 0.0))
                node_eff_next = float(kpi_eff_next.get(addr, 0.0))

                stake_share_pct_cur  = (node_eff_cur  / total_effective_cur  * 100.0) if total_effective_cur  > 0 else 0.0
                stake_share_pct_next = (node_eff_next / total_effective_next * 100.0) if total_effective_next > 0 else 0.0
                delivered_slots_share_pct = (ok / total_slots_epoch * 100.0) if total_slots_epoch > 0 else 0.0
                bad_slots_pct_node = (bad_abs / max(1, int(slots)) * 100.0)

                # main solo per label (ok usare mapping vivo)
                node = self.overflows[addr]
                main = self.mains[node.main_id]

                # ricompense/fee (snapshot + cumulati)
                coinbase_epoch = ok * reward_per_block
                fees_epoch = float(kpi_fees_m.get(main.address, 0.0))
                coinbase_cum = ok_cum_map[addr] * reward_per_block
                fees_cum_main = fees_cum_by_main[main.address]

                # penalità (debug/scomposizione)
                pdb = kpi_pdebug.get(addr, {})
                pen_gained   = float(pdb.get("gained", 0.0))
                pen_recovery = float(pdb.get("recovery", 0.0))
                pen_tol      = float(pdb.get("tol", 0.0))
                resid_prev   = float(pdb.get("residual_prev", kpi_resid.get(addr, 0)))
                resid_next   = float(pdb.get("residual_next", kpi_resid.get(addr, 0)))

                # invalid malevoli: quota realizzati/planned e share sugli invalid totali
                forced_planned  = int(kpi_forced_planned.get(addr, 0))
                forced_realized = int(kpi_forced_realized.get(addr, 0))
                forced_share_of_invalid = (forced_realized / invalid * 100.0) if invalid > 0 else 0.0

                rows.append({
                    "epoch": kpi["epoch"],
                    "overflow": addr,
                    "main": main.address,

                    # assegnazione
                    "assigned_slots": int(slots),
                    "share_assigned_slots": round((int(slots) / S) * 100, 3),

                    # esiti slot
                    "ok": ok, "skipped": skipped, "invalid": invalid, "delayed": delayed,
                    "bad_slots_abs": int(bad_abs),
                    "bad_slots_pct_node": round(bad_slots_pct_node, 2),

                    # stake effettivo (corrente vs prossimo)
                    "effective_stake": node_eff_cur,
                    "stake_share_pct": round(stake_share_pct_cur, 4),
                    "effective_stake_next": node_eff_next,
                    "stake_share_next_pct": round(stake_share_pct_next, 4),

                    # delivery share
                    "delivered_slots_share_pct": round(delivered_slots_share_pct, 4),

                    # ricompense/fee
                    "coinbase_epoch_TKG": round(coinbase_epoch, 9),
                    "fees_epoch_TKR_at_main": round(fees_epoch, 9),
                    "reward_epoch_total_value_TKG_only": round(coinbase_epoch, 9),
                    "coinbase_cum_TKG": round(coinbase_cum, 9),
                    "fees_cum_TKR_at_main": round(fees_cum_main, 9),

                    # penalità (snapshot epoca)
                    "penalty_slots_residual": int(kpi_resid.get(addr, 0)),
                    "penalty_score": float(kpi_score.get(addr, 0.0)),

                    # invalid malevoli (diagnostica)
                    "forced_invalid_slots_planned": forced_planned,
                    "forced_invalid_slots_realized": forced_realized,
                    "forced_invalid_share_pct_of_invalid": round(forced_share_of_invalid, 4),

                    # breakdown penalità (debug)
                    "penalty_gained": pen_gained,
                    "penalty_recovery": pen_recovery,
                    "penalty_tol": pen_tol,
                    "residual_prev_dbg": resid_prev,
                    "residual_next_dbg": resid_next,
                })

        return pd.DataFrame(rows)

# Simulations

## Simulation 1

This simulation runs for 200 epochs with a large mempool of randomly generated transactions per slot.
- Initial stake distribution: Node 1 = 40%, Node 2 = 30%, Node 3 = 30% (delegations at epoch 0).
- Behavior: all three nodes are honest; every assigned slot is produced as ok (no skips/invalid/delayed).
- Expected outcome: the scheduler allocates slots ≈ proportionally to stake (≈40/30/30); coinbase (TKG) and fees (TKR) follow the same proportions up to small stochastic noise; no penalties accumulate.

In [ ]:
# --- SIMULAZIONE: setup + run -------------------------------------------------

# 1) Istanzia il simulatore con i parametri desiderati
sim = TakamakaSim(
    slots_per_epoch = 24_000,
    eep_fraction    = 1/3,
    penalty_factor  = 1.0,
    recovery_factor = 1.0,
    tolerated_bad_ratio = 0.01,
    reward_per_block = 0.0001,
    seed = 42,
    slash_per_invalid = 0.0,
    penalty_alpha = 1e-3,
    w_skipped = 1.0,
    w_invalid = 2.0,
    w_delayed = 1.5,
    txs_per_slot_mean = 0.05,
    max_txs_per_block = 1,

    # scenografia transazioni/penalità
    tx_invalid_rate = 0.05,           # % di TX cattive generabili (da epoca 1 in poi)
    auto_mempool_epoch0 = False,      # epoca 0: nessuna TX auto
    auto_mempool_after_epoch0 = True, # epoche >=1: TX auto
    malicious_invalid_share = 0.05    # ~5% degli slot assegnati al nodo malevolo diventa invalid/epoca
)

# 2) Registra i MAIN (wallet dei main)
m1 = MainAddress(address="MAIN_1", green=0.0, red=0.0)
m2 = MainAddress(address="MAIN_2", green=0.0, red=0.0)
m3 = MainAddress(address="MAIN_3", green=0.0, red=0.0)
sim.register_main(m1)
sim.register_main(m2)
sim.register_main(m3)

# 3) Overflow nodes (tutti onesti all’inizio: p_invalid=0,
#    gli unici esiti "di rete" possibili sono skip/delay se li imposti tu)
o1 = OverflowNode(address="OV_1", green=0.0, red=0.0,
                  p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)
o2 = OverflowNode(address="OV_2", green=0.0, red=0.0,
                  p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)
o3 = OverflowNode(address="OV_3", green=0.0, red=0.0,
                  p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)

# collega gli overflow ai rispettivi main e indicizzali nel simulatore
m1.register_overflow(o1); sim.index_overflow(o1)
m2.register_overflow(o2); sim.index_overflow(o2)
m3.register_overflow(o3); sim.index_overflow(o3)

# 4) Stakeholders (40/30/30)
s1 = Wallet(address="STK_1", green=400.0, red=50.0)
s2 = Wallet(address="STK_2", green=300.0, red=50.0)
s3 = Wallet(address="STK_3", green=300.0, red=50.0)
sim.register_wallet(s1)
sim.register_wallet(s2)
sim.register_wallet(s3)

# 5) Wallet “clienti” per generare le transazioni (mittenti)
w1 = Wallet(address="W_1", green=400.0, red=50.0)
w2 = Wallet(address="W_2", green=300.0, red=50.0)
w3 = Wallet(address="W_3", green=300.0, red=50.0)
sim.register_wallet(w1)
sim.register_wallet(w2)
sim.register_wallet(w3)

# mittenti = W_* ; destinatari = stakeholders
sim.set_tx_participants(
    senders=["W_1", "W_2", "W_3"],
    recipients=["STK_1", "STK_2", "STK_3"]
)

# 6) Deleghe all’epoca 0 (slot 0) — in epoca 0 non vogliamo penalty
sim.queue_bet(from_addr="STK_1", to_main_id="MAIN_1", amount=400.0, when_slot=0)
sim.queue_bet(from_addr="STK_2", to_main_id="MAIN_2", amount=300.0, when_slot=0)
sim.queue_bet(from_addr="STK_3", to_main_id="MAIN_3", amount=300.0, when_slot=0)

# 7) Simulazione di 200 epoche
for ep in range(200):
    sim.simulate_one_epoch()

# 8) Esporta risultati

summary_baseline = sim.summary_all_epochs_df()
summary_baseline.to_csv("./data/scenario1_baseline.csv", index=False)

In [ ]:
sim = TakamakaSim(
    slots_per_epoch=SLOTS_PER_EPOCH,
    seed=RNG_SEED
)

# main (wallet)
m1 = MainAddress(address="MAIN_1", green=0.0, red=0.0)
m2 = MainAddress(address="MAIN_2", green=0.0, red=0.0)
m3 = MainAddress(address="MAIN_3", green=0.0, red=0.0)

# registra come main
sim.register_main(m1)
sim.register_main(m2)
sim.register_main(m3)

# overflow onesti
o1 = OverflowNode(address="OV_1", green=0.0, red=0.0, p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)
o2 = OverflowNode(address="OV_2", green=0.0, red=0.0, p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)
o3 = OverflowNode(address="OV_3", green=0.0, red=0.0, p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)

# registra overflow e collega ai main e indicizza
m1.register_overflow(o1); sim.index_overflow(o1)
m2.register_overflow(o2); sim.index_overflow(o2)
m3.register_overflow(o3); sim.index_overflow(o3)

# stakeholders con stake 40/30/30
s1 = Wallet(address="STK_1", green=400.0, red=50.0)
s2 = Wallet(address="STK_2", green=300.0, red=50.0)
s3 = Wallet(address="STK_3", green=300.0, red=50.0)

# registra stakeholder come wallet
sim.register_wallet(s1)
sim.register_wallet(s2)
sim.register_wallet(s3)

# Wallets to simulate transactions
w1 = Wallet(address="W_1", green=400.0, red=50.0)
w2 = Wallet(address="W_2", green=300.0, red=50.0)
w3 = Wallet(address="W_3", green=300.0, red=50.0)

sim.register_wallet(w1)
sim.register_wallet(w2)
sim.register_wallet(w3)

sim.set_tx_participants(senders=["W_1","W_2","W_3"], recipients=["STK_1","STK_2","STK_3"])

# deleghe epoch 0 (slot 0)
sim.queue_bet(from_addr="STK_1", to_main_id="MAIN_1", amount=400.0, when_slot=0)
sim.queue_bet(from_addr="STK_2", to_main_id="MAIN_2", amount=300.0, when_slot=0)
sim.queue_bet(from_addr="STK_3", to_main_id="MAIN_3", amount=300.0, when_slot=0)

for _ in range(50):
    sim.simulate_one_epoch()

# Un solo DataFrame alla fine
summary_baseline = sim.summary_all_epochs_df()
summary_baseline.to_csv("./data/scenario1_baseline.csv", index=False)


: 

: 

## Simulation 2

Same setup as Simulation 1 (stake 40/30/30, rich mempool, 200 epochs), but with transient malicious behavior for Node 1:
- Epochs 0–49: Node 1 behaves maliciously (e.g., corrupt/underpaid transactions via p_bad_tx=1.0), so its blocks are marked invalid, fees are not collected, and penalties increase its residual, reducing its effective stake and slot share.
- Epochs 50–199: Node 1 returns to honest behavior; recovery reduces residual penalties over time and its effective stake/slot share gradually restores.
- Expected outcome: during 0–49 Node 1’s valid blocks and rewards drop sharply; from 50–199 they recover progressively, illustrating long-term penalization and recovery dynamics.

In [9]:
# --- SIMULAZIONE: setup + run -------------------------------------------------

# 1) Istanzia il simulatore con i parametri desiderati
sim = TakamakaSim(
    # --- PARAMETRI UFFICIALI (dai doc) ---
    slots_per_epoch = 24_000,   # ufficiale
    eep_fraction    = 1/3,      # EEP = 8.000 su 24.000 (ufficiale)
    penalty_factor  = 2.0,      # PF = 2 (ufficiale)
    recovery_factor = 1.0,      # RF = 1 (ufficiale)
    tolerated_bad_ratio = 0.02, # 2% tolleranza (valore di riferimento nei doc)

    # --- SCELTE DI MODELLO / TESI (non fissate dai doc) ---
    reward_per_block = 0.0001,  # scelta di simulazione (coinbase per blocco)
    seed = 42,                  # scelta di simulazione (PRNG)
    slash_per_invalid = 0.0,    # opzionale: niente slashing nel modello
    penalty_alpha = 1e-3,       # scala dell’impatto della penalità sui pesi (tuning)
    w_skipped = 1.0,            # pesi qualità esiti: tuning
    w_invalid = 2.0,            # tuning
    w_delayed = 1.5,            # tuning

    # --- TRAFFICO TX (solo per lo scenario, non parte del protocollo) ---
    txs_per_slot_mean = 1,
    max_txs_per_block = 1,

    tx_invalid_rate = 0.05,           # % TX cattive generabili (da epoca 1 in poi) - scenario
    auto_mempool_epoch0 = False,      # epoca 0: niente TX auto - scenario
    auto_mempool_after_epoch0 = True, # epoche >=1: TX auto - scenario
    malicious_invalid_share = 0.05    # quota slot invalid forzati sul nodo malevolo - scenario
)

# 2) Registra i MAIN (wallet dei main)
m1 = MainAddress(address="MAIN_1", green=0.0, red=0.0)
m2 = MainAddress(address="MAIN_2", green=0.0, red=0.0)
m3 = MainAddress(address="MAIN_3", green=0.0, red=0.0)
sim.register_main(m1)
sim.register_main(m2)
sim.register_main(m3)

# 3) Overflow nodes (tutti onesti all’inizio: p_invalid=0,
#    gli unici esiti "di rete" possibili sono skip/delay se li imposti tu)
o1 = OverflowNode(address="OV_1", green=0.0, red=0.0,
                  p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)
o2 = OverflowNode(address="OV_2", green=0.0, red=0.0,
                  p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)
o3 = OverflowNode(address="OV_3", green=0.0, red=0.0,
                  p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)

# collega gli overflow ai rispettivi main e indicizzali nel simulatore
m1.register_overflow(o1); sim.index_overflow(o1)
m2.register_overflow(o2); sim.index_overflow(o2)
m3.register_overflow(o3); sim.index_overflow(o3)

# 4) Stakeholders (40/30/30)
s1 = Wallet(address="STK_1", green=400.0, red=50.0)
s2 = Wallet(address="STK_2", green=300.0, red=50.0)
s3 = Wallet(address="STK_3", green=300.0, red=50.0)
sim.register_wallet(s1)
sim.register_wallet(s2)
sim.register_wallet(s3)

# 5) Wallet “clienti” per generare le transazioni (mittenti)
w1 = Wallet(address="W_1", green=400.0, red=50.0)
w2 = Wallet(address="W_2", green=300.0, red=50.0)
w3 = Wallet(address="W_3", green=300.0, red=50.0)
sim.register_wallet(w1)
sim.register_wallet(w2)
sim.register_wallet(w3)

# mittenti = W_* ; destinatari = stakeholders
sim.set_tx_participants(
    senders=["W_1", "W_2", "W_3"],
    recipients=["STK_1", "STK_2", "STK_3"]
)

# 6) Deleghe all’epoca 0 (slot 0) — in epoca 0 non vogliamo penalty
sim.queue_bet(from_addr="STK_1", to_main_id="MAIN_1", amount=400.0, when_slot=0)
sim.queue_bet(from_addr="STK_2", to_main_id="MAIN_2", amount=300.0, when_slot=0)
sim.queue_bet(from_addr="STK_3", to_main_id="MAIN_3", amount=300.0, when_slot=0)

# 7) Simulazione di 200 epoche
for ep in range(200):
    # Epoche 0–49: OV_1 si comporta malevolmente (tenta di includere tx cattive),
    # ma il numero di slot invalidi per epoca è limitato da malicious_invalid_share.
    o1.p_bad_tx = 1.0 if ep < 50 else 0.0

    # Nota: p_invalid resta 0. Gli "invalid" nascono SOLO dal comportamento malevolo.
    sim.simulate_one_epoch()

# 8) Esporta risultati

summary_malicious = sim.summary_all_epochs_df()
summary_malicious.to_csv("./data/scenario2_malicious.csv", index=False)